In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# url = ('https://newsapi.org/v2/everything?'
#        #'country=us&'
#        'q=ConEdison&'
#        'apiKey=c942149366d547a996be6242c9d0e7c4')

In [3]:
url = 'https://newyork.cbslocal.com/tag/power-outage/'
res = requests.get(url)

In [4]:
soup = BeautifulSoup(res.content)

In [5]:
soup.find('div', {'class':"title-wrapper"})

<div class="title-wrapper"><strong class="title">Breaking News</strong><em class="description">Notre Dame Cathedral In Paris On Fire</em></div>

In [6]:
for i in soup.find_all('div', {'class':"title-wrapper"}):
    print(i)

<div class="title-wrapper"><strong class="title">Breaking News</strong><em class="description">Notre Dame Cathedral In Paris On Fire</em></div>
<div class="title-wrapper"><strong class="title">Menu</strong></div>
<div class="title-wrapper"><strong class="title">CBS2 Videos</strong></div>
<div class="title-wrapper"><strong class="title">CBS Sports Live</strong></div>
<div class="title-wrapper"><strong class="title">Giants</strong><em class="description">New York Giants</em></div>
<div class="title-wrapper"><strong class="title">Jets</strong><em class="description">New York Jets</em></div>
<div class="title-wrapper"><strong class="title">Yanks</strong><em class="description">New York Yankees</em></div>
<div class="title-wrapper"><strong class="title">Mets</strong><em class="description">New York Mets</em></div>
<div class="title-wrapper"><strong class="title">Knicks</strong><em class="description">New York Knicks</em></div>
<div class="title-wrapper"><strong class="title">Nets</strong><e

In [ ]:
Consumer API keys
IRBBt2ZSwW819m6Sdgd6Kdzmu (API key)

rrCgkiFmOFXrqcqRk3Rny2J4HL8TasDPDJgvRpcFgGok6QADeZ (API secret key)


Regenerate
Access token & access token secret
71166867-hlflZCK6Bf2T5Yfw4oQ2mfsapZ9MTmXKSy9EC61Ys (Access token)NEW

pqBXwi0sHVOYzU63UQH7M7jpX2RnKNTTeGWEq4fKQTzTo (Access token secret)

In [53]:
import json

# Enter your keys/secrets as strings in the following fields
credentials = {}  
credentials['CONSUMER_KEY'] = 'IRBBt2ZSwW819m6Sdgd6Kdzmu' 
credentials['CONSUMER_SECRET'] = 'rrCgkiFmOFXrqcqRk3Rny2J4HL8TasDPDJgvRpcFgGok6QADeZ'
credentials['ACCESS_TOKEN'] = '71166867-hlflZCK6Bf2T5Yfw4oQ2mfsapZ9MTmXKSy9EC61Ys'  
credentials['ACCESS_SECRET'] = 'pqBXwi0sHVOYzU63UQH7M7jpX2RnKNTTeGWEq4fKQTzTo'




In [54]:
import tweepy as tw

auth = tw.OAuthHandler(credentials['CONSUMER_KEY'], credentials['CONSUMER_SECRET'])
auth.set_access_token(credentials['ACCESS_TOKEN'], credentials['ACCESS_SECRET'])

api = tw.API(auth)

In [51]:
from twitterscraper import query_tweets
import time

In [52]:
data_2 = []
for tweet in query_tweets('conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-12-31&src=typd', 10):
    print(tweet.text)
    tweets = {}
    tweets['userid'] = tweet.user.encode('utf-8')
    tweets['id'] = tweet.id
    tweets['date'] = tweet.timestamp
    tweets['tweet'] = tweet.text
    data_2.append(tweets)
    time.sleep(2)
    
df_2 = pd.DataFrame(data_2)

INFO: queries: ['conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-12-31&src=typd since:2006-03-21 until:2006-11-14', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-12-31&src=typd since:2006-11-14 until:2007-07-11', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-12-31&src=typd since:2007-07-11 until:2008-03-06', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-12-31&src=typd since:2008-03-06 until:2008-10-30', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-12-31&src=typd since:2008-10-30 until:2009-06-26', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-12-31&src=typd since:2009-06-26 until:2010-02-20', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-12-31&src=typd since:2010-02-20 until:2010-10-16', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-12-31&src=typd since:2010-10-16 until:2011-06-12', 'conedison%20"power%20outage"%20since%3A2017-01-

INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets for conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-12-31&src=typd%20since%3A2018-08-20%20until%3A2019-04-16.
INFO: Got 0 tweets (0 new).


In [15]:
import pandas as pd
df = pd.DataFrame(data)

In [16]:
df.head()

,date,id,tweet,userid
0,2017-12-30 14:06:17,947106570776842241,.@ConEdison is responding to a power outage in...,b'NotifyNYC'
1,2017-12-30 02:57:31,946938269601816576,@ConEdison Hi! We are experiencing a power out...,b'Hilary_Weaver'
2,2017-12-29 04:04:27,946592726073663491,Power outage in Brooklyn Heights and @ConEdiso...,b'KarenEdelman'
3,2017-12-28 20:05:52,946472287179558913,.@ConEdison is responding to a power outage in...,b'NotifyNYC'
4,2017-12-22 01:47:32,944021553200680960,@ConEdison Any updates on what caused the powe...,b'EddieLuc'


In [17]:
df.shape

(400, 4)

In [19]:
df.tail()

,date,id,tweet,userid
395,2017-11-29 02:53:59,935703357402046465,.@ConEdison ConEd is responding to a power out...,b'NotifyNYC'
396,2017-11-23 02:09:30,933517833530691586,Accident at intersection 62nd Street/65th Plac...,b'SNIPNYC'
397,2017-11-23 01:55:23,933514283429842944,UPDATE: there is a localized power outage in a...,b'NYPD104Pct'
398,2017-11-23 00:34:08,933493834704654336,NotifyNYC: .ConEdison ConEd responding to a p...,b'EastCoast911IA'
399,2017-11-23 00:29:43,933492725227425793,.@ConEdison ConEd responding to a power outag...,b'NotifyNYC'


In [20]:
df.to_csv('11.23.17_to_12_31_17.csv')

In [23]:
data_2 = []
for tweet in query_tweets('conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-11-23&src=typd', 10):
    print(tweet.text)
    print(tweet.timestamp)
    tweets = {}
    tweets['userid'] = tweet.user.encode('utf-8')
    tweets['id'] = tweet.id
    tweets['date'] = tweet.timestamp
    tweets['tweet'] = tweet.text
    data_2.append(tweets)
    time.sleep(2)
    
df_2 = pd.DataFrame(data_2)

INFO: queries: ['conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-11-23&src=typd since:2006-03-21 until:2006-11-14', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-11-23&src=typd since:2006-11-14 until:2007-07-11', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-11-23&src=typd since:2007-07-11 until:2008-03-06', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-11-23&src=typd since:2008-03-06 until:2008-10-30', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-11-23&src=typd since:2008-10-30 until:2009-06-26', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-11-23&src=typd since:2009-06-26 until:2010-02-20', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-11-23&src=typd since:2010-02-20 until:2010-10-16', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-11-23&src=typd since:2010-10-16 until:2011-06-12', 'conedison%20"power%20outage"%20since%3A2017-01-

INFO: Got 380 tweets (20 new).
INFO: Got 20 tweets for conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-11-23&src=typd%20since%3A2006-03-21%20until%3A2006-11-14.
INFO: Got 400 tweets (20 new).


@ConEdison power outage in Queens Village between 90th Ave & Braddock
2017-11-21 03:43:10
NotifyNYC: .ConEdison ConEd  responding to a power outage in the Cambria Heights section of Jamaica, Queens, including ZIP codes 11411, 11412, 11413, 11422. Report service loss: 1-800-75-CONED. Multilingual & ASL link: http://on.nyc.gov/22lyehC .
2017-11-17 02:16:31
.@ConEdison ConEd  responding to a power outage in the Cambria Heights section of Jamaica, Queens, including ZIP codes 11411, 11412, 11413, 11422. Report service loss: 1-800-75-CONED. Multilingual & ASL link: http://on.nyc.gov/22lyehC .
2017-11-17 02:09:56
Power outage affecting residents around Pine Avenue. @ConEdison has an emergency crew on scene working to restore power. pic.twitter.com/4PeSrmNHOy
2017-11-13 22:43:05
NotifyNYC: .ConEdison ConEd is responding to a power outage in  Bronx ZIP 10466.  Report service loss: 1-800-75-CO… http://on.nyc.gov/22lyehC .
2017-11-09 00:48:08
.@ConEdison ConEd is responding to a power outage in  

Just a power outage in 10308. I see reports on the outage map already so no big deal.
2017-10-30 03:57:22
.@ConEdison is responding to a power outage in QN ZIPs 11412 & 11429. Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-10-30 03:54:58
@ConEdison @ConEdSolutions tried to report power outage and got an error message. Please fix (website and power). Thanks! pic.twitter.com/iW5E8HlgpN
2017-10-24 16:21:52
@ConEdison power outage in the 400 block of Pratt avenue 
2017-10-23 08:53:33
NotifyNYC: .ConEdison is responding to a power outage in SI ZIPs 10312 & 10314.  Report service loss: 1-800-75-CON… http://on.nyc.gov/22lyehC .
2017-10-22 23:22:25
.@ConEdison is responding to a power outage in SI ZIPs 10312 & 10314.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-10-22 23:19:12
NotifyNYC: .ConEdison is responding to a power outage in Coney Island, BK, ZIP: 11214.  Report service loss: 1-800… http://on.nyc.gov/22lyehC .
2017-10-13 17:31:30
.@ConEdi

Power outage affecting residents around Pine Avenue. @ConEdison has an emergency crew on scene working to restore power. pic.twitter.com/4PeSrmNHOy
2017-11-13 22:43:05
NotifyNYC: .ConEdison ConEd is responding to a power outage in  Bronx ZIP 10466.  Report service loss: 1-800-75-CO… http://on.nyc.gov/22lyehC .
2017-11-09 00:48:08
.@ConEdison ConEd is responding to a power outage in  Bronx ZIP 10466.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-11-09 00:44:53
NotifyNYC: .ConEdison is responding to a power outage in Queens, ZIP: 11355.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-11-08 02:33:13
.@ConEdison is responding to a power outage in Queens, ZIP: 11355.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-11-08 02:29:11
NotifyNYC: .ConEdison is responding to a power outage in BX ZIPs 10461 & 10462.  Report service loss: 1-800-75-CON… http://on.nyc.gov/22lyehC .
2017-11-03 14:57:04
.@ConEdison is responding to a 

NotifyNYC: .ConEdison is responding to a power outage in SI ZIPs 10312 & 10314.  Report service loss: 1-800-75-CON… http://on.nyc.gov/22lyehC .
2017-10-22 23:22:25
.@ConEdison is responding to a power outage in SI ZIPs 10312 & 10314.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-10-22 23:19:12
NotifyNYC: .ConEdison is responding to a power outage in Coney Island, BK, ZIP: 11214.  Report service loss: 1-800… http://on.nyc.gov/22lyehC .
2017-10-13 17:31:30
.@ConEdison is responding to a power outage in Coney Island, BK, ZIP: 11214.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-10-13 17:29:35
NotifyNYC: .ConEdison is responding to a power outage in Queens, ZIPs: 11412 & 11433.  Report service loss: 1-800-… http://on.nyc.gov/22lyehC 
2017-10-12 16:35:51
@ConEdison power outage in Queens Village between 90th Ave & Braddock
2017-11-21 03:43:10
NotifyNYC: .ConEdison ConEd  responding to a power outage in the Cambria Heights section of Jamaica,

.@ConEdison ConEd is responding to a power outage in  Bronx ZIP 10466.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-11-09 00:44:53
NotifyNYC: .ConEdison is responding to a power outage in Queens, ZIP: 11355.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-11-08 02:33:13
.@ConEdison is responding to a power outage in Queens, ZIP: 11355.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-11-08 02:29:11
NotifyNYC: .ConEdison is responding to a power outage in BX ZIPs 10461 & 10462.  Report service loss: 1-800-75-CON… http://on.nyc.gov/22lyehC .
2017-11-03 14:57:04
.@ConEdison is responding to a power outage in BX ZIPs 10461 & 10462.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-11-03 14:48:18
NotifyNYC: .ConEdison is responding to a power outage in QN ZIPs 11412 & 11429. Report service loss: 1-800-75-CONE… http://on.nyc.gov/22lyehC .
2017-10-30 05:48:12
Just a power outage in 10308. I see reports

NotifyNYC: .ConEdison is responding to a power outage in Coney Island, BK, ZIP: 11214.  Report service loss: 1-800… http://on.nyc.gov/22lyehC .
2017-10-13 17:31:30
.@ConEdison is responding to a power outage in Coney Island, BK, ZIP: 11214.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-10-13 17:29:35
NotifyNYC: .ConEdison is responding to a power outage in Queens, ZIPs: 11412 & 11433.  Report service loss: 1-800-… http://on.nyc.gov/22lyehC 
2017-10-12 16:35:51
@ConEdison power outage in Queens Village between 90th Ave & Braddock
2017-11-21 03:43:10
NotifyNYC: .ConEdison ConEd  responding to a power outage in the Cambria Heights section of Jamaica, Queens, including ZIP codes 11411, 11412, 11413, 11422. Report service loss: 1-800-75-CONED. Multilingual & ASL link: http://on.nyc.gov/22lyehC .
2017-11-17 02:16:31
.@ConEdison ConEd  responding to a power outage in the Cambria Heights section of Jamaica, Queens, including ZIP codes 11411, 11412, 11413, 11422. Report

.@ConEdison is responding to a power outage in Queens, ZIP: 11355.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-11-08 02:29:11
NotifyNYC: .ConEdison is responding to a power outage in BX ZIPs 10461 & 10462.  Report service loss: 1-800-75-CON… http://on.nyc.gov/22lyehC .
2017-11-03 14:57:04
.@ConEdison is responding to a power outage in BX ZIPs 10461 & 10462.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-11-03 14:48:18
NotifyNYC: .ConEdison is responding to a power outage in QN ZIPs 11412 & 11429. Report service loss: 1-800-75-CONE… http://on.nyc.gov/22lyehC .
2017-10-30 05:48:12
Just a power outage in 10308. I see reports on the outage map already so no big deal.
2017-10-30 03:57:22
.@ConEdison is responding to a power outage in QN ZIPs 11412 & 11429. Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-10-30 03:54:58
@ConEdison @ConEdSolutions tried to report power outage and got an error message. Please fix (website 

NotifyNYC: .ConEdison is responding to a power outage in Queens, ZIPs: 11412 & 11433.  Report service loss: 1-800-… http://on.nyc.gov/22lyehC 
2017-10-12 16:35:51
@ConEdison power outage in Queens Village between 90th Ave & Braddock
2017-11-21 03:43:10
NotifyNYC: .ConEdison ConEd  responding to a power outage in the Cambria Heights section of Jamaica, Queens, including ZIP codes 11411, 11412, 11413, 11422. Report service loss: 1-800-75-CONED. Multilingual & ASL link: http://on.nyc.gov/22lyehC .
2017-11-17 02:16:31
.@ConEdison ConEd  responding to a power outage in the Cambria Heights section of Jamaica, Queens, including ZIP codes 11411, 11412, 11413, 11422. Report service loss: 1-800-75-CONED. Multilingual & ASL link: http://on.nyc.gov/22lyehC .
2017-11-17 02:09:56
Power outage affecting residents around Pine Avenue. @ConEdison has an emergency crew on scene working to restore power. pic.twitter.com/4PeSrmNHOy
2017-11-13 22:43:05
NotifyNYC: .ConEdison ConEd is responding to a power ou

In [30]:
df_2.head()

,date,id,tweet,userid
0,2017-11-21 03:43:10,932816633344774151,@ConEdison power outage in Queens Village betw...,b'YserbiusG'
1,2017-11-17 02:16:31,931345273426726915,NotifyNYC: .ConEdison ConEd responding to a p...,b'EastCoast911IA'
2,2017-11-17 02:09:56,931343615376359424,.@ConEdison ConEd responding to a power outag...,b'NotifyNYC'
3,2017-11-13 22:43:05,930204398717988867,Power outage affecting residents around Pine A...,b'OssiningPolice'
4,2017-11-09 00:48:08,928423930603491328,NotifyNYC: .ConEdison ConEd is responding to a...,b'EastCoast911IA'


In [29]:
df_2.tail()

,date,id,tweet,userid
395,2017-10-22 23:22:25,922241761925312512,NotifyNYC: .ConEdison is responding to a power...,b'EastCoast911IA'
396,2017-10-22 23:19:12,922240954765103104,.@ConEdison is responding to a power outage in...,b'NotifyNYC'
397,2017-10-13 17:31:30,918891962845028352,NotifyNYC: .ConEdison is responding to a power...,b'EastCoast911IA'
398,2017-10-13 17:29:35,918891478851751942,.@ConEdison is responding to a power outage in...,b'NotifyNYC'
399,2017-10-12 16:35:51,918515570647162881,NotifyNYC: .ConEdison is responding to a power...,b'EastCoast911IA'


In [31]:
df_2.to_csv('10.12.2017_to_11.21.2017')

In [32]:
data_3 = []
for tweet in query_tweets('conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-10-12&src=typd', 30):
    print(tweet.text)
    print(tweet.timestamp)
    tweets = {}
    tweets['userid'] = tweet.user.encode('utf-8')
    tweets['id'] = tweet.id
    tweets['date'] = tweet.timestamp
    tweets['tweet'] = tweet.text
    data_3.append(tweets)
    time.sleep(2)
    
df_3 = pd.DataFrame(data_3)

INFO: queries: ['conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-10-12&src=typd since:2006-03-21 until:2006-11-14', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-10-12&src=typd since:2006-11-14 until:2007-07-11', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-10-12&src=typd since:2007-07-11 until:2008-03-06', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-10-12&src=typd since:2008-03-06 until:2008-10-30', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-10-12&src=typd since:2008-10-30 until:2009-06-26', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-10-12&src=typd since:2009-06-26 until:2010-02-20', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-10-12&src=typd since:2010-02-20 until:2010-10-16', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-10-12&src=typd since:2010-10-16 until:2011-06-12', 'conedison%20"power%20outage"%20since%3A2017-01-

INFO: Got 20 tweets for conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-10-12&src=typd%20since%3A2011-06-12%20until%3A2012-02-06.
INFO: Got 380 tweets (20 new).
INFO: Got 400 tweets (20 new).


@ConEdison power outage - no AC - 16th St between 5th & 6th South Slope 11215
It's 90 degrees out...and 85 inside apt
2017-09-24 19:41:39
NotifyNYC: .ConEdison ConEd is responding to a power outage in BX ZIP 10467.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-09-24 11:53:03
.@ConEdison ConEd is responding to a power outage in BX ZIP 10467.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-09-24 11:50:35
NotifyNYC: .ConEdison ConEd is responding to a power outage in Staten Island, ZIP 10306.  Report service loss: 1-8… http://on.nyc.gov/22lyehC 
2017-09-09 16:12:10
.@ConEdison ConEd is responding to a power outage in Staten Island, ZIP 10306.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC 
2017-09-09 16:10:05
@ConEdison completely blocking Cabrini from 181st to 187th at noon with no permit, no power outage.   @NYC_DOT ?  180 Cabrini, Manhattan.pic.twitter.com/QvJdhLL5vT
2017-09-06 16:17:46
NotifyNYC: .ConEdison is responding

Sorry to hear this. You can report power outage to ConEdison by calling 800-752-6633 or online: http://bit.ly/2qxmSfQ 
2017-08-24 17:45:47
NotifyNYC: .ConEdison ConEd is responding to a power outage in Huguenot, SI, ZIPs: 10312 and 10314.  Report service loss: 1-800-75-CONED.
2017-08-23 04:26:17
.@ConEdison ConEd is responding to a power outage in Huguenot, SI, ZIPs: 10312 and 10314.  Report service loss: 1-800-75-CONED.
2017-08-23 04:26:02
NotifyNYC: .ConEdison ConEd is responding to a power outage in Annadale, SI, ZIPs: 10309.  Report service loss: 1-800-75-CONED.
2017-08-23 04:16:08
.@ConEdison ConEd is responding to a power outage in Annadale, SI, ZIPs: 10309.  Report service loss: 1-800-75-CONED.
2017-08-23 04:10:20
@ConEdison power outage - no AC - 16th St between 5th & 6th South Slope 11215
It's 90 degrees out...and 85 inside apt
2017-09-24 19:41:39
NotifyNYC: .ConEdison ConEd is responding to a power outage in BX ZIP 10467.  Report service loss: 1-800-75-CONED. http://on.nyc.go

@ConEdison power outage in middle village queens - several buildings. Eta on restoration?
2017-08-31 12:54:22
Power outage in my neighborhood, so no #GameOfThrones for me.  Get on it, @ConEdison pic.twitter.com/M2q4OLXEku
2017-08-28 02:34:28
NotifyNYC: ConEdison ConEd is responding to a power outage in Seaview, SI, ZIPs: 10314.  Report service loss: 1-800-75-CONED.
2017-08-26 16:38:14
@ConEdison ConEd is responding to a power outage in Seaview, SI, ZIPs: 10314.  Report service loss: 1-800-75-CONED.
2017-08-26 16:35:34
@CONEDISON FYI There is a power outage on Elwood Ave Staten Island N.Y. 10314 #Heartlandvillage #Newspringville
2017-08-26 00:23:07
Sorry to hear this. You can report power outage to ConEdison by calling 800-752-6633 or online: http://bit.ly/2qxmSfQ 
2017-08-24 17:45:47
NotifyNYC: .ConEdison ConEd is responding to a power outage in Huguenot, SI, ZIPs: 10312 and 10314.  Report service loss: 1-800-75-CONED.
2017-08-23 04:26:17
.@ConEdison ConEd is responding to a power outa

NotifyNYC: .ConEdison is responding to a power outage in BX ZIPs 10461 & 10465.  Report service loss: 1-800-75-CON… http://on.nyc.gov/22lyehC .
2017-09-03 18:23:11
.@ConEdison is responding to a power outage in BX ZIPs 10461 & 10465.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-09-03 18:20:33
NotifyNYC: .ConEdison responding to a power outage in QN ZIPs: 11368, 11373, & 11374.  Report service loss: 1-800-… http://on.nyc.gov/22lyehC 
2017-08-31 13:18:24
.@ConEdison responding to a power outage in QN ZIPs: 11368, 11373, & 11374.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC 
2017-08-31 13:16:12
@ConEdison power outage in middle village queens - several buildings. Eta on restoration?
2017-08-31 12:54:22
Power outage in my neighborhood, so no #GameOfThrones for me.  Get on it, @ConEdison pic.twitter.com/M2q4OLXEku
2017-08-28 02:34:28
NotifyNYC: ConEdison ConEd is responding to a power outage in Seaview, SI, ZIPs: 10314.  Report service loss: 1-80

.@ConEdison ConEd is responding to a power outage in BX ZIP 10467.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-09-24 11:50:35
NotifyNYC: .ConEdison ConEd is responding to a power outage in Staten Island, ZIP 10306.  Report service loss: 1-8… http://on.nyc.gov/22lyehC 
2017-09-09 16:12:10
.@ConEdison ConEd is responding to a power outage in Staten Island, ZIP 10306.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC 
2017-09-09 16:10:05
@ConEdison completely blocking Cabrini from 181st to 187th at noon with no permit, no power outage.   @NYC_DOT ?  180 Cabrini, Manhattan.pic.twitter.com/QvJdhLL5vT
2017-09-06 16:17:46
NotifyNYC: .ConEdison is responding to a power outage in BX ZIPs 10461 & 10465.  Report service loss: 1-800-75-CON… http://on.nyc.gov/22lyehC .
2017-09-03 18:23:11
.@ConEdison is responding to a power outage in BX ZIPs 10461 & 10465.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-09-03 18:20:33
NotifyNYC: .ConE

.@ConEdison ConEd is responding to a power outage in Huguenot, SI, ZIPs: 10312 and 10314.  Report service loss: 1-800-75-CONED.
2017-08-23 04:26:02
NotifyNYC: .ConEdison ConEd is responding to a power outage in Annadale, SI, ZIPs: 10309.  Report service loss: 1-800-75-CONED.
2017-08-23 04:16:08
.@ConEdison ConEd is responding to a power outage in Annadale, SI, ZIPs: 10309.  Report service loss: 1-800-75-CONED.
2017-08-23 04:10:20
@ConEdison power outage - no AC - 16th St between 5th & 6th South Slope 11215
It's 90 degrees out...and 85 inside apt
2017-09-24 19:41:39
NotifyNYC: .ConEdison ConEd is responding to a power outage in BX ZIP 10467.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-09-24 11:53:03
.@ConEdison ConEd is responding to a power outage in BX ZIP 10467.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-09-24 11:50:35
NotifyNYC: .ConEdison ConEd is responding to a power outage in Staten Island, ZIP 10306.  Report service loss: 1

NotifyNYC: ConEdison ConEd is responding to a power outage in Seaview, SI, ZIPs: 10314.  Report service loss: 1-800-75-CONED.
2017-08-26 16:38:14
@ConEdison ConEd is responding to a power outage in Seaview, SI, ZIPs: 10314.  Report service loss: 1-800-75-CONED.
2017-08-26 16:35:34
@CONEDISON FYI There is a power outage on Elwood Ave Staten Island N.Y. 10314 #Heartlandvillage #Newspringville
2017-08-26 00:23:07
Sorry to hear this. You can report power outage to ConEdison by calling 800-752-6633 or online: http://bit.ly/2qxmSfQ 
2017-08-24 17:45:47
NotifyNYC: .ConEdison ConEd is responding to a power outage in Huguenot, SI, ZIPs: 10312 and 10314.  Report service loss: 1-800-75-CONED.
2017-08-23 04:26:17
.@ConEdison ConEd is responding to a power outage in Huguenot, SI, ZIPs: 10312 and 10314.  Report service loss: 1-800-75-CONED.
2017-08-23 04:26:02
NotifyNYC: .ConEdison ConEd is responding to a power outage in Annadale, SI, ZIPs: 10309.  Report service loss: 1-800-75-CONED.
2017-08-23 04

.@ConEdison is responding to a power outage in BX ZIPs 10461 & 10465.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-09-03 18:20:33
NotifyNYC: .ConEdison responding to a power outage in QN ZIPs: 11368, 11373, & 11374.  Report service loss: 1-800-… http://on.nyc.gov/22lyehC 
2017-08-31 13:18:24
.@ConEdison responding to a power outage in QN ZIPs: 11368, 11373, & 11374.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC 
2017-08-31 13:16:12
@ConEdison power outage in middle village queens - several buildings. Eta on restoration?
2017-08-31 12:54:22
Power outage in my neighborhood, so no #GameOfThrones for me.  Get on it, @ConEdison pic.twitter.com/M2q4OLXEku
2017-08-28 02:34:28
NotifyNYC: ConEdison ConEd is responding to a power outage in Seaview, SI, ZIPs: 10314.  Report service loss: 1-800-75-CONED.
2017-08-26 16:38:14
@ConEdison ConEd is responding to a power outage in Seaview, SI, ZIPs: 10314.  Report service loss: 1-800-75-CONED.
2017-08-26 16:35

In [34]:
df_3.head()

,date,id,tweet,userid
0,2017-09-24 19:41:39,912039347221880832,@ConEdison power outage - no AC - 16th St betw...,b'victoriajbd'
1,2017-09-24 11:53:03,911921418803310592,NotifyNYC: .ConEdison ConEd is responding to a...,b'EastCoast911IA'
2,2017-09-24 11:50:35,911920798268579842,.@ConEdison ConEd is responding to a power out...,b'NotifyNYC'
3,2017-09-09 16:12:10,906550810473062408,NotifyNYC: .ConEdison ConEd is responding to a...,b'EastCoast911IA'
4,2017-09-09 16:10:05,906550283798491138,.@ConEdison ConEd is responding to a power out...,b'NotifyNYC'


In [35]:
df_3.tail()

,date,id,tweet,userid
395,2017-08-24 17:45:47,900776161344716800,Sorry to hear this. You can report power outag...,b'nyc311'
396,2017-08-23 04:26:17,900212574121316354,NotifyNYC: .ConEdison ConEd is responding to a...,b'EastCoast911IA'
397,2017-08-23 04:26:02,900212512569917442,.@ConEdison ConEd is responding to a power out...,b'NotifyNYC'
398,2017-08-23 04:16:08,900210021576921089,NotifyNYC: .ConEdison ConEd is responding to a...,b'EastCoast911IA'
399,2017-08-23 04:10:20,900208558599528450,.@ConEdison ConEd is responding to a power out...,b'NotifyNYC'


In [36]:
df_3.to_csv('08.23.17_to_09.24.17.csv')

In [37]:
!ls

08.23.17_to_09.24.17.csv    Client_Project_John.ipynb
10.12.2017_to_11.21.2017    Client_Project_Kevin.ipynb
11.23.17_to_12_31_17.csv    Untitled.pdf
Client_Project_Ixchel.ipynb


In [40]:
data_4 = []
for tweet in query_tweets('conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-08-23&src=typd', 30):
    print(tweet.text)
    print(tweet.timestamp)
    tweets = {}
    tweets['userid'] = tweet.user.encode('utf-8')
    tweets['id'] = tweet.id
    tweets['date'] = tweet.timestamp
    tweets['tweet'] = tweet.text
    data_4.append(tweets)
    time.sleep(2)
    
df_4 = pd.DataFrame(data_4)

INFO: queries: ['conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-08-23&src=typd since:2006-03-21 until:2006-11-14', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-08-23&src=typd since:2006-11-14 until:2007-07-11', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-08-23&src=typd since:2007-07-11 until:2008-03-06', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-08-23&src=typd since:2008-03-06 until:2008-10-30', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-08-23&src=typd since:2008-10-30 until:2009-06-26', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-08-23&src=typd since:2009-06-26 until:2010-02-20', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-08-23&src=typd since:2010-02-20 until:2010-10-16', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-08-23&src=typd since:2010-10-16 until:2011-06-12', 'conedison%20"power%20outage"%20since%3A2017-01-

INFO: Got 380 tweets (20 new).
INFO: Got 20 tweets for conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-08-23&src=typd%20since%3A2014-09-18%20until%3A2015-05-14.
INFO: Got 400 tweets (20 new).


@ConEdison just had a power outage on 3rd ave between 33rd and 34th
2017-08-22 13:27:41
NotifyNYC: .ConEdison is responding to a power outage in SI ZIPs 10309 & 10312.  Report service loss: 1-800-75-CON… http://on.nyc.gov/22lyehC .
2017-08-18 23:58:43
.@ConEdison is responding to a power outage in SI ZIPs 10309 & 10312.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-08-18 23:56:50
All power has been restored after power outage in Dyker Heights this morning! @ConEdison http://ow.ly/q2Yk30eqr2H 
2017-08-15 15:52:30
Good morning. We are aware of a power outage at 2400 Nostrand Avenue. Please DM us for information and provide your contact number. ~SKS
2017-08-15 15:17:04
NotifyNYC: .ConEdison is responding to a power outage in Dyker Loop, BK; ZIP code: 11228. Report service loss: 1-8… http://on.nyc.gov/22lyehC 
2017-08-15 10:23:41
.@ConEdison is responding to a power outage in Dyker Loop, BK; ZIP code: 11228. Report service loss: 1-800-75-CONED. http://on.nyc.gov/22

@ConEdison just had a power outage on 3rd ave between 33rd and 34th
2017-08-22 13:27:41
NotifyNYC: .ConEdison is responding to a power outage in SI ZIPs 10309 & 10312.  Report service loss: 1-800-75-CON… http://on.nyc.gov/22lyehC .
2017-08-18 23:58:43
.@ConEdison is responding to a power outage in SI ZIPs 10309 & 10312.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-08-18 23:56:50
All power has been restored after power outage in Dyker Heights this morning! @ConEdison http://ow.ly/q2Yk30eqr2H 
2017-08-15 15:52:30
Good morning. We are aware of a power outage at 2400 Nostrand Avenue. Please DM us for information and provide your contact number. ~SKS
2017-08-15 15:17:04
NotifyNYC: .ConEdison is responding to a power outage in Dyker Loop, BK; ZIP code: 11228. Report service loss: 1-8… http://on.nyc.gov/22lyehC 
2017-08-15 10:23:41
.@ConEdison is responding to a power outage in Dyker Loop, BK; ZIP code: 11228. Report service loss: 1-800-75-CONED. http://on.nyc.gov/22

@ConEdison just had a power outage on 3rd ave between 33rd and 34th
2017-08-22 13:27:41
NotifyNYC: .ConEdison is responding to a power outage in SI ZIPs 10309 & 10312.  Report service loss: 1-800-75-CON… http://on.nyc.gov/22lyehC .
2017-08-18 23:58:43
.@ConEdison is responding to a power outage in SI ZIPs 10309 & 10312.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-08-18 23:56:50
All power has been restored after power outage in Dyker Heights this morning! @ConEdison http://ow.ly/q2Yk30eqr2H 
2017-08-15 15:52:30
Good morning. We are aware of a power outage at 2400 Nostrand Avenue. Please DM us for information and provide your contact number. ~SKS
2017-08-15 15:17:04
NotifyNYC: .ConEdison is responding to a power outage in Dyker Loop, BK; ZIP code: 11228. Report service loss: 1-8… http://on.nyc.gov/22lyehC 
2017-08-15 10:23:41
.@ConEdison is responding to a power outage in Dyker Loop, BK; ZIP code: 11228. Report service loss: 1-800-75-CONED. http://on.nyc.gov/22

@ConEdison just had a power outage on 3rd ave between 33rd and 34th
2017-08-22 13:27:41
NotifyNYC: .ConEdison is responding to a power outage in SI ZIPs 10309 & 10312.  Report service loss: 1-800-75-CON… http://on.nyc.gov/22lyehC .
2017-08-18 23:58:43
.@ConEdison is responding to a power outage in SI ZIPs 10309 & 10312.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-08-18 23:56:50
All power has been restored after power outage in Dyker Heights this morning! @ConEdison http://ow.ly/q2Yk30eqr2H 
2017-08-15 15:52:30
Good morning. We are aware of a power outage at 2400 Nostrand Avenue. Please DM us for information and provide your contact number. ~SKS
2017-08-15 15:17:04
NotifyNYC: .ConEdison is responding to a power outage in Dyker Loop, BK; ZIP code: 11228. Report service loss: 1-8… http://on.nyc.gov/22lyehC 
2017-08-15 10:23:41
.@ConEdison is responding to a power outage in Dyker Loop, BK; ZIP code: 11228. Report service loss: 1-800-75-CONED. http://on.nyc.gov/22

@ConEdison just had a power outage on 3rd ave between 33rd and 34th
2017-08-22 13:27:41
NotifyNYC: .ConEdison is responding to a power outage in SI ZIPs 10309 & 10312.  Report service loss: 1-800-75-CON… http://on.nyc.gov/22lyehC .
2017-08-18 23:58:43
.@ConEdison is responding to a power outage in SI ZIPs 10309 & 10312.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-08-18 23:56:50
All power has been restored after power outage in Dyker Heights this morning! @ConEdison http://ow.ly/q2Yk30eqr2H 
2017-08-15 15:52:30
Good morning. We are aware of a power outage at 2400 Nostrand Avenue. Please DM us for information and provide your contact number. ~SKS
2017-08-15 15:17:04
NotifyNYC: .ConEdison is responding to a power outage in Dyker Loop, BK; ZIP code: 11228. Report service loss: 1-8… http://on.nyc.gov/22lyehC 
2017-08-15 10:23:41
.@ConEdison is responding to a power outage in Dyker Loop, BK; ZIP code: 11228. Report service loss: 1-800-75-CONED. http://on.nyc.gov/22

@ConEdison just had a power outage on 3rd ave between 33rd and 34th
2017-08-22 13:27:41
NotifyNYC: .ConEdison is responding to a power outage in SI ZIPs 10309 & 10312.  Report service loss: 1-800-75-CON… http://on.nyc.gov/22lyehC .
2017-08-18 23:58:43
.@ConEdison is responding to a power outage in SI ZIPs 10309 & 10312.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-08-18 23:56:50
All power has been restored after power outage in Dyker Heights this morning! @ConEdison http://ow.ly/q2Yk30eqr2H 
2017-08-15 15:52:30
Good morning. We are aware of a power outage at 2400 Nostrand Avenue. Please DM us for information and provide your contact number. ~SKS
2017-08-15 15:17:04
NotifyNYC: .ConEdison is responding to a power outage in Dyker Loop, BK; ZIP code: 11228. Report service loss: 1-8… http://on.nyc.gov/22lyehC 
2017-08-15 10:23:41
.@ConEdison is responding to a power outage in Dyker Loop, BK; ZIP code: 11228. Report service loss: 1-800-75-CONED. http://on.nyc.gov/22

@ConEdison just had a power outage on 3rd ave between 33rd and 34th
2017-08-22 13:27:41
NotifyNYC: .ConEdison is responding to a power outage in SI ZIPs 10309 & 10312.  Report service loss: 1-800-75-CON… http://on.nyc.gov/22lyehC .
2017-08-18 23:58:43
.@ConEdison is responding to a power outage in SI ZIPs 10309 & 10312.  Report service loss: 1-800-75-CONED. http://on.nyc.gov/22lyehC .
2017-08-18 23:56:50
All power has been restored after power outage in Dyker Heights this morning! @ConEdison http://ow.ly/q2Yk30eqr2H 
2017-08-15 15:52:30
Good morning. We are aware of a power outage at 2400 Nostrand Avenue. Please DM us for information and provide your contact number. ~SKS
2017-08-15 15:17:04
NotifyNYC: .ConEdison is responding to a power outage in Dyker Loop, BK; ZIP code: 11228. Report service loss: 1-8… http://on.nyc.gov/22lyehC 
2017-08-15 10:23:41
.@ConEdison is responding to a power outage in Dyker Loop, BK; ZIP code: 11228. Report service loss: 1-800-75-CONED. http://on.nyc.gov/22

In [41]:
df_4.shape

(400, 4)

In [42]:
df_4.head()

,date,id,tweet,userid
0,2017-08-22 13:27:41,899986434379190272,@ConEdison just had a power outage on 3rd ave ...,b'Jillemmy'
1,2017-08-18 23:58:43,898695688480882688,NotifyNYC: .ConEdison is responding to a power...,b'EastCoast911IA'
2,2017-08-18 23:56:50,898695213488549888,.@ConEdison is responding to a power outage in...,b'NotifyNYC'
3,2017-08-15 15:52:30,897486165422608388,All power has been restored after power outage...,b'bklyner'
4,2017-08-15 15:17:04,897477246797131778,Good morning. We are aware of a power outage a...,b'ConEdison'


In [43]:
df_4.tail()

,date,id,tweet,userid
395,2017-07-30 04:46:19,891520305432285185,NotifyNYC: .ConEdison is responding to a power...,b'EastCoast911IA'
396,2017-07-30 04:44:02,891519731198570497,.@ConEdison is responding to a power outage in...,b'NotifyNYC'
397,2017-07-30 04:11:20,891511502792007681,Sooo power outage in Brooklyn @ConEdison hopef...,b'LorealT'
398,2017-07-29 19:48:40,891385002356858882,Really @ConEdison its not even a hot day & we ...,b'Brooklyynnboo'
399,2017-07-28 19:46:26,891022051426930689,NotifyNYC: .ConEdison ConEd is responding to a...,b'EastCoast911IA'


In [44]:
df_4.to_csv('07.28.17_to_08.22.17.csv')

In [45]:
!ls

07.28.17_to_08.22.17.csv    Client_Project_Ixchel.ipynb
08.23.17_to_09.24.17.csv    Client_Project_John.ipynb
10.12.2017_to_11.21.2017    Client_Project_Kevin.ipynb
11.23.17_to_12_31_17.csv    Untitled.pdf


In [54]:
data_5 = []
for tweet in query_tweets('conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-08-22&src=typd', 1):
    print(tweet.text)
    print(tweet.timestamp)
    tweets = {}
    tweets['userid'] = tweet.user.encode('utf-8')
    tweets['id'] = tweet.id
    tweets['date'] = tweet.timestamp
    tweets['tweet'] = tweet.text
    data_5.append(tweets)
    time.sleep(2)
    
df_5 = pd.DataFrame(data_5)

INFO: queries: ['conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-08-22&src=typd since:2006-03-21 until:2006-11-14', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-08-22&src=typd since:2006-11-14 until:2007-07-11', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-08-22&src=typd since:2007-07-11 until:2008-03-06', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-08-22&src=typd since:2008-03-06 until:2008-10-30', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-08-22&src=typd since:2008-10-30 until:2009-06-26', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-08-22&src=typd since:2009-06-26 until:2010-02-20', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-08-22&src=typd since:2010-02-20 until:2010-10-16', 'conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-08-22&src=typd since:2010-10-16 until:2011-06-12', 'conedison%20"power%20outage"%20since%3A2017-01-

INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets for conedison%20"power%20outage"%20since%3A2017-01-01%20until%3A2017-08-22&src=typd%20since%3A2012-10-02%20until%3A2013-05-28.
INFO: Got 0 tweets (0 new).
